## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os

# Import API key
#from config import g_key
from dotenv import load_dotenv
load_dotenv()
Google_api_key = os.getenv("Google_api_key")
print(type(Google_api_key))

# Configure gmaps API key
gmaps.configure(api_key=Google_api_key)

<class 'str'>


In [6]:
# 1. Import the WeatherPy_database.csv file. 
#data_to_load = os.path.join("Weather_Database", "WeatherPy_Database.csv")
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Puerto Ayora,EC,-0.7393,-90.3518,78.76,85,91,5.01,Clouds
1,1,Buraydah,SA,26.3260,43.9750,99.27,27,90,8.81,Clouds
2,2,College,US,64.8569,-147.8028,57.29,94,75,3.44,Fog
3,3,Lompoc,US,34.6391,-120.4579,86.68,73,0,6.91,Clear
4,4,Busselton,AU,-33.6500,115.3333,58.77,74,62,20.49,Clouds


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [9]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Puerto Ayora,EC,-0.7393,-90.3518,78.76,85,91,5.01,Clouds
3,3,Lompoc,US,34.6391,-120.4579,86.68,73,0,6.91,Clear
5,5,Caravelas,BR,-17.7125,-39.2481,72.68,88,62,10.16,Clouds
11,11,Veraval,IN,20.9000,70.3667,81.16,82,70,13.96,Clouds
12,12,Guerrero Negro,MX,27.9769,-114.0611,82.72,46,10,7.85,Clear
...,...,...,...,...,...,...,...,...,...,...
671,671,Akdepe,TM,42.0551,59.3788,87.89,32,95,13.06,Clouds
672,672,Santa Maria Del Oro,MX,25.9333,-105.3667,83.19,32,32,7.14,Clouds
674,674,Sibenik,HR,43.7272,15.9058,85.66,45,0,7.74,Clear
676,676,La Asuncion,VE,11.0333,-63.8628,83.93,74,100,23.02,Clouds


In [34]:
# 4a. Determine if there are any empty rows.
#preferred_cities_df.count()
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [31]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_cities_df = preferred_cities_df.dropna()

In [28]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Puerto Ayora,EC,78.76,Clouds,-0.7393,-90.3518,
3,Lompoc,US,86.68,Clear,34.6391,-120.4579,
5,Caravelas,BR,72.68,Clouds,-17.7125,-39.2481,
11,Veraval,IN,81.16,Clouds,20.9000,70.3667,
12,Guerrero Negro,MX,82.72,Clear,27.9769,-114.0611,
14,Faanui,PF,77.68,Rain,-16.4833,-151.7500,
17,Labuhan,ID,77.07,Clouds,-6.8844,112.2051,
22,Avarua,CK,73.45,Clouds,-21.2078,-159.7750,
29,Albany,US,83.68,Clouds,42.6001,-73.9662,
35,Bo Phloi,TH,77.95,Clouds,14.3236,99.5161,


In [29]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": Google_api_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"] 
    except (IndexError):
        print("Hotel not found... skipping.")
# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
-----------------------------
Data Retrieval Complete      
-----------------------------


In [50]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
clean_hotel_df = hotel_df.replace('', np.nan, inplace = True)
clean_hotel_df = hotel_df.dropna()
#clean_hotel_df.isnull().sum()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Puerto Ayora,EC,78.76,Clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
3,Lompoc,US,86.68,Clear,34.6391,-120.4579,"Holiday Inn Express Lompoc, an IHG Hotel"
5,Caravelas,BR,72.68,Clouds,-17.7125,-39.2481,Pousada dos Navegantes
11,Veraval,IN,81.16,Clouds,20.9000,70.3667,Lords Inn Somnath
12,Guerrero Negro,MX,82.72,Clear,27.9769,-114.0611,"Casa Laguna, Bed & Breakfast"
...,...,...,...,...,...,...,...
669,Iracoubo,GF,82.90,Clouds,5.4802,-53.2011,Le Gîte d’Iracoubo
670,Kamaishi,JP,75.40,Clouds,39.2667,141.8833,Hotel Route Inn Kamaishi
672,Santa Maria Del Oro,MX,83.19,Clouds,25.9333,-105.3667,Hotel Palacio Del Sol
674,Sibenik,HR,85.66,Clear,43.7272,15.9058,Hotel Jadran


In [62]:
# 8a. Create the output File (CSV)
output_data_file = ("WeatherPy_vacation.csv")
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [63]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [61]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))